In [1]:
import pm4py 
bpmn_model = pm4py.read_bpmn('BPMN.bpmn')

net, im , fm = pm4py.convert_to_petri_net(bpmn_model)


In [2]:
from core_simulator import SimulationEngine, log_analyzer
engine = SimulationEngine(net, im, fm)
business_log = engine.run_simulation()

In [3]:
print(business_log[0]) # datetime.datetime(1970, 4, 27, 0, 31, 12, tzinfo=datetime.timezone.utc)

{'attributes': {'concept:name': 'Application_652823628', 'LoanGoal': 'Existing loan takeover', 'ApplicationType': 'New credit', 'RequestedAmount': 20000.0}, 'events': [{'concept:name': 'A_Create Application', 'org:resource': 'user_2', 'time:timestamp': datetime.datetime(2025, 1, 1, 9, 7, 39, 725047), 'lifecycle:transition': 'complete'}, {'concept:name': 'W_Complete application', 'org:resource': 'user_3', 'time:timestamp': datetime.datetime(2025, 1, 1, 9, 8, 52, 508958), 'lifecycle:transition': 'complete'}, {'concept:name': 'W_Call after offers', 'org:resource': 'user_1', 'time:timestamp': datetime.datetime(2025, 1, 1, 9, 10, 54, 885617), 'lifecycle:transition': 'complete'}, {'concept:name': 'W_Call after offers', 'org:resource': 'user_2', 'time:timestamp': datetime.datetime(2025, 1, 1, 9, 19, 53, 105977), 'lifecycle:transition': 'complete'}, {'concept:name': 'A_Cancelled', 'org:resource': 'user_2', 'time:timestamp': datetime.datetime(2025, 1, 1, 9, 23, 47, 472326), 'lifecycle:transitio

In [4]:
len(business_log)

10

In [5]:
analyzer = log_analyzer(business_log)

analyzer.print_analysis()

=== PROCESS SIMULATION ANALYSIS ===

Trace Length | Max: 50 | Min: 8 | Avg: 21.60
Total Time (min) | Max: 265.64 | Min: 30.81 | Avg: 114.70
Inter-Arrival (min) | Max: 67.58 | Min: 51.83 | Avg: 59.54

Activity Performance (min):
 - W_Complete application    | Avg:   5.53 | Max:   9.70
 - W_Call after offers       | Avg:   4.87 | Max:   9.09
 - A_Cancelled               | Avg:   5.86 | Max:   9.26
 - O_Cancelled               | Avg:   5.87 | Max:   9.61
 - A_Submitted               | Avg:   6.99 | Max:   8.44
 - W_Handle leads            | Avg:   5.48 | Max:   9.49
 - A_Concept                 | Avg:   5.36 | Max:   9.93
 - A_Accepted                | Avg:   5.26 | Max:   8.27
 - O_Create Offer            | Avg:   6.09 | Max:   8.83
 - O_Created                 | Avg:   5.23 | Max:   8.95
 - O_Sent (mail and online)  | Avg:   5.98 | Max:   9.91
 - W_Validate application    | Avg:   6.69 | Max:   9.91
 - O_Accepted                | Avg:   4.94 | Max:   7.72
 - A_Pending                 | 